In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.datasets import mnist

In [7]:
# Importing the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [10]:
train_X[:,0].shape

(60000, 28)

In [ ]:
W1 = np.random.randn(10,784)
b1 = np.random.randn(10,1)
W2 = np.random.randn(10,10)
b2 = np.random.randn(10,1)

def relu(Z):
    return np.maximum(0,Z)
def softmax(Z):
    return np.exp(Z)/np.sum(np.exp(Z))
Z1 = W1.dot(train_X ) + b1
A1 = relu(Z1)
Z2 = W2.dot(A1) + b2
A2 = softmax(Z2)

In [ ]:
def one_hot(y):
    one_hot = np.zeros((10, y.shape[0]))
    for i in range(y.shape[0]):
        one_hot[y[i],i] = 1
    return one_hot

In [ ]:
one_hot_Y = one_hot(train_y)
m = train_y.size
dZ2 = A2 - one_hot_Y
dW2 = 1/m * dZ2.dot(A1.T)
db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
dZ1 = W2.T.dot(dZ2) * (A1 > 0)  # relu derivative
dW1 = 1/m * dZ1.dot(train_X.T)
db1 = 1/m * np.sum(dZ1,2)